# DP0.2 Notebook 17 Galaxy Photometry

<img align="left" src = https://project.lsst.org/sites/default/files/Rubin-O-Logo_0.png width=250 style="padding: 10px" alt="Rubin Observatory logo, a graphical representation of turning stars into data.">
<br>
Contact author(s): <i>Christina Williams</i> <br>
Last verified to run: <i>2024-02-10</i> <br>
LSST Science Pipelines version: Weekly <i>2024_50</i> <br>
Container Size: <i>medium</i> <br>
Targeted learning level: <i>beginner</i> <br>

In [ ]:
#%load_ext pycodestyle_magic
#%flake8_on
#import logging
#logging.getLogger("flake8").setLevel(logging.FATAL)

**Description:** Explore the available measurements of galaxy photometry produced by the LSST pipelines and their applications.

**Skills:** Basic understanding of photometric measurement methods typically used to characterize galaxies.

**LSST Data Products:** objectTable

**Packages:** No new LSST packages are introduced here. The notebook will make use of various `lsst` packages introduced in earlier tutorial notebooks.

**Credit:**
This notebook benefitted from earlier exploration of simulated data and notebook development by Melissa Graham and Dan Taranu, and helpful discussions with Jim Bosch. 

**Get Support:**
Find DP0-related documentation and resources at <a href="https://dp0.lsst.io">dp0.lsst.io</a>.
Questions are welcome as new topics in the 
<a href="https://community.lsst.org/c/support/dp0">Support - Data Preview 0 Category</a> 
of the Rubin Community Forum. 
Rubin staff will respond to all questions posted there.

## 1. Introduction

This notebook will teach the user about the automated photometry measurements that are measured on the deepCoadd images and appear in the Object Catalog as part of the LSST pipelines data products. The focus will be on galaxies (and thus is distinct from the demonstrations in Tutorial Notebooks <a href="https://dp0-2.lsst.io/_static/nb_html/DP02_12a_PSF_Data_Products.html">12a</a> and <a href="https://dp0-2.lsst.io/_static/nb_html/DP02_12b_PSF_Science_Demo.html">12b</a>   about the PSF photometry.) 

Please note that these explanations are specific to DP0.2: the objectTable contents and definitions are still subject to change for DP1. 


### 1.1 Types of photometry measurements that exist in the Object Table

Schema for the object catalog for DP0.2: https://dm.lsst.org/sdm_schemas/browser/dp02.html#Object

Numerous photometry measurements are produced by the LSST Pipelines. Two types of photometry are there: The first are total fluxes are those that integrate the total light coming from objects, corrected for loss due to the blurring effects of PSF or seeing, including "Composite Model" (`cModel`) fluxes. The second category characterize the relative brightness of objects for specific regions of objects (i.e. are not corrected to be total fluxes) but can be useful for measuring accurate light profiles or accurate colors.

### 1.1.1 Total fluxes


#### Composite Model (CModel) fluxes

Similar in nature to those measured for SDSS: 
https://www.sdss3.org/dr8/algorithms/magnitudes.php#cmodel

In short, it is the linear combination of the best fit exponential (disk or D) and de Vaucouleurs (bulge or B) profiles. 

```
<f>_cModelFlux    : Flux from the final cmodel fit. Forced on <f>-band.
<f>_cModelFluxErr : Uncertainty of <f>_cModelFlux
<f>_cModel_flag   : Failure flag for <f>_cModelFlux
```

For most cases the "fixed" cModel photometry (as listed above) are preferred to that measured with more degrees of freedom labeled `<f>_free_cModelFlux`. The difference is that the fixed ones above uses a reference band (recorded as `refBand` in the schema) where the galaxy is well detected to determine the other parameters, which are then fixed when fitting for the flux in the other bands. The `_free_cModelFlux` measurement allows all parameters to be free and independent in each filter. The fixed `_cModelFlux` measurements are generally recommended for galaxy science applications where total flux measurements are needed (e.g. for intrinsic luminosity or mass).

### 1.1.2 Apparent fluxes (i.e. not corrected to the total flux)
 
#### Kron fluxes

A decent summary of Kron fluxes <a href="https://ned.ipac.caltech.edu/level5/March05/Graham/Graham2_6.html">in the NED documentation</a>. The aperture used for the fluxes is 2.5 x R1 where R1 is the luminosity weighted radius (also called "first moment"; Kron et al. 1980).

```
<f>_kronFlux      : Flux from Kron Flux algorithm. Measured on <f> g-band.
<f>_kronFluxErr   : Uncertainty of <f>_kronFlux.
<f>_kronFlux_flag : Failure flag for <f>_kronFlux.
```

The Kron radius, `<f>_kronRad`, is also available. In this case of LSST pipeline output, the Kron flux is not corrected for light that is emitted outside of the Kron aperture. While in many cases it will collect the majority of light, it will not be as accurate as the cModel for science cases requiring total flux.


#### Aperture fluxes
This contains the enclosed flux inside a given aperture (they are not corrected to total fluxes using an aperture correction that accounts for the flux falling outside the aperture). Fixed aperture size refers to the aperture radius in pixels.

```
<f>_ap<pix>Flux     : Flux within <pix>-pixel aperture. Forced on <f>-band.
<f>_ap<pix>FluxErr  : Uncertainty of <f>_ap<pix>Flux.
<f>_ap<pix>FluxFlag : Failure flag for <f>_ap<pix>Flux.
```

For DP0.2, the apertures are 3, 6, 9, 12, 17, 25, 35, 50, and 70 pixels. In the column name, apertures are `03`, `06`, `09`, `12`, and so on. While aperture fluxes are not corrected for the loss outside the aperture, if the aperture size is much larger than the galaxy size then it will approximate the total flux of the galaxy. The general application of these measurements are for measuring radial profiles (see Section 4 below).

#### GaaP fluxes

These are the Gaussian-aperture-and-PSF flux that is defined in <a href="https://ui.adsabs.harvard.edu/abs/2008A%26A...482.1053K/abstract">Kuijken et al. 2008</a>. The main goal of this method is to measure accurate colors while accounting for the different spatial resolution between filters. This is sometimes achieved in other datasets by convolving all images to the largest PSF, but this process of PSF-matching is computationally very time consuming for large images, thus motivating GaaP as a faster alternative. It is not a measure of total flux in a filter. Several apertures are available. For photometric redshifts, and other analysis where accurate colors are important, it is the GaaP fluxes with optimal aperture that should be used. 

**Optimal**

```
<f>_gaapOptimalFlux    : GaaP flux with optimal aperture after multiplying the seeing by 1.15. Forced on <f>-band.
<f>_gaapOptimalFluxErr : Uncertainty of <f>_gaapOptimalFlux.
```

**PSF**

```
<f>_gaapPsfFlux    : GaaP flux with PSF aperture after multiplying the seeing by 1.15. Forced on <f>-band.
<f>_gaapPsfFluxErr : Uncertainty of <f>_gaapPsfFlux.
```

**Aperture**

```
<f>_gaap<ap>Flux    : GaaP flux with <ap> aperture after multiplying the seeing aperture. Forced on <f>-band.
<f>_gaap<ap>FluxErr : Uncertainty of <f>_gaap<ap>Flux.
```

Where the seeing apertures are 0.5, 0.7, 1.0, 1.5, 2.5, and 3.0 arcseconds. In the column name `<ap>` appears as `0p5`, `0p7`, etc.


#### PSF fluxes

Fluxes derived using the model point-spread function (PSF) of the image. Since PSF fluxes are designed for unresolved sources (not the majority of galaxies) this notebook will not explore science applications for this type of photometry. Interested users can find more information in the PSF Tutorial notebooks <a href="https://dp0-2.lsst.io/_static/nb_html/DP02_12a_PSF_Data_Products.html">12a</a> and <a href="https://dp0-2.lsst.io/_static/nb_html/DP02_12b_PSF_Science_Demo.html">12b</a>.

```
<f>_psfFlux      : Flux derived from linear least-squares fit of PSF model. Forced on <f>-band.
<f>_psfFluxErr   : Uncertainty of <f>_psfFlux.
<f>_psfFlux_flag : Failure flag for <f>_psfFlux.

```


### 1.2 Import packages


`numpy` is a fundamental package for scientific computing with arrays in Python
(<a href="https://numpy.org">numpy.org</a>).

`matplotlib` is a comprehensive library for creating static, animated, and
interactive visualizations in Python 
(<a href="https://matplotlib.org/">matplotlib.org</a>; 
<a href="https://matplotlib.org/stable/gallery/index.html">matplotlib gallery</a>).

`scipy.stats` contains statistical packages, this notebook will use binned_statistic to calculate running medians.

From the `lsst` package, modules for accessing the TAP service, and image display functions from `afw` are imported (<a href="https://pipelines.lsst.io/">pipelines.lsst.io</a>). Also import some geometric functions to help plot photometric apertures.

From the `pyvo` package, import some functions that will enable using the image cutout tool.

Finally, from `astropy` and `photutils` import packages to enable plotting images and drawing shapes on images with WCS information.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import binned_statistic

import lsst.geom as geom
import lsst.resources
import lsst.afw.display as afwDisplay
from lsst.afw.image import ExposureF
from lsst.rsp import get_tap_service
from lsst.afw.fits import MemFileManager
import lsst.afw.geom.ellipses as ellipses

from pyvo.dal.adhoc import DatalinkResults, SodaQuery
import uuid

from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.wcs import WCS
from photutils.aperture import SkyCircularAperture, SkyEllipticalAperture


### 1.3 Define functions and parameters


#### 1.3.1 Define some aesthetic plotting defaults

In [ ]:
afwDisplay.setDefaultBackend('matplotlib')

params = {'axes.labelsize': 18,
          'font.size': 18,
          'legend.fontsize': 12,
          'xtick.major.width': 2,
          'xtick.minor.width': 1,
          'xtick.major.size': 10,
          'xtick.minor.size': 4,
          'xtick.direction': 'in',
          'xtick.top': True,
          'lines.linewidth': 2,
          'axes.linewidth': 2,
          'axes.labelweight': 2,
          'axes.titleweight': 2,
          'ytick.major.width': 2,
          'ytick.minor.width': 1,
          'ytick.major.size': 10,
          'ytick.minor.size': 4,
          'ytick.direction': 'in',
          'ytick.right': True,
          'figure.figsize': [6, 6],
          'figure.facecolor': 'White'
          }
plt.style.use('tableau-colorblind10')

plt.rcParams.update(params)

#### 1.3.2 Define a shortcut function to generate an image cutout using the cutout tool
Further information about the cutout tool can be found in <a href="https://dp0-2.lsst.io/_static/nb_html/DP02_13a_Image_Cutout_SciDemo.html">tutorial notebook 13a</a>.

In [ ]:
def make_image_cutout_new(tap_service, ra, dec, cutout_size=0.01):
    """Wrapper function to generate a cutout using the cutout tool

   Parameters
    ----------
    tap_service : an instance of the TAP service
    ra, dec : 'float'
        the ra and dec of the cutout center
    cutout_size : 'float', optional
        edge length in degrees of the cutout

   Returns
    -------
    mem : 'object'
        An lsst.afw.fits._fits.MemFileManager object containing the
        cutout image bytes returned by the cutout tool
    """

    spherePoint = geom.SpherePoint(ra*geom.degrees, dec*geom.degrees)
    coord = SkyCoord(ra=ra*u.degree, dec=dec*u.degree, frame='icrs')

    band = 'i'

    query = "SELECT access_format, access_url, dataproduct_subtype, " + \
            "lsst_patch, lsst_tract, lsst_band, s_ra, s_dec " + \
            "FROM ivoa.ObsCore WHERE dataproduct_type = 'image' " + \
            "AND obs_collection = 'LSST.DP02' " + \
            "AND dataproduct_subtype = 'lsst.deepCoadd_calexp' " + \
            "AND lsst_band = " + "'" + str(band) + "' " + \
            "AND CONTAINS(POINT('ICRS', " + str(coord.ra.value) + \
            ", " + str(coord.dec.value) + "), " + \
            "s_region) = 1"

    results = tap_service.search(query)

    # Get datalink
    dataLinkUrl = results[0].getdataurl()
    auth_session = service._session
    dl = DatalinkResults.from_result_url(dataLinkUrl,
                                         session=auth_session)

    # from_resource: creates a instance from
    # a number of records and a Datalink Resource.
    sq = SodaQuery.from_resource(dl,
                                 dl.get_adhocservice_by_id("cutout-sync"),
                                 session=auth_session)

    sq.circle = (spherePoint.getRa().asDegrees() * u.deg,
                 spherePoint.getDec().asDegrees() * u.deg,
                 cutout_size * u.deg)
    cutout_bytes = sq.execute_stream().read()
    mem = MemFileManager(len(cutout_bytes))
    mem.setData(cutout_bytes, len(cutout_bytes))

    return mem

## 2. Integrated photometry of galaxies using the LSST pipeline

This section will explore integrated photometry measurements, and provide some guidance for which are optimal for certain science applications using galaxies. 


### 2.1 Initialize the TAP service

In [ ]:
service = get_tap_service("tap")

### 2.2 Object table photometry measurements

First, see what is available in the object catalog by querying the tap_schema columns, and printing all the parameters available related to "Flux" measured in the i-band (as an example). The return shows also errors and flags associated with the photometric measurements outlined in section 1.1. 

In [ ]:
query = "SELECT column_name, datatype, description, unit " \
        "FROM tap_schema.columns " \
        "WHERE table_name = 'dp02_dc2_catalogs.Object'"

results = service.search(query).to_table()

In [ ]:
search_string = 'Flux'
band = 'i_'
for cname in results['column_name']:
    if (cname.find(search_string) > -1) and (cname.find(band) > -1):
        print(cname)

#### 2.3 Compare photometric measurements

### 2.3.1 Identify a galaxy sample to investigate photometry

Below, query the DP0.2 objectTable for a selection of photometric measurements. This section will focus on 4 measurements: total flux from cModel (sum of bulge and disk sersic compoents fitted to the galaxy); and apparent fluxes measured as Kron (typically includes more than 90% of intrinsic light), GaaP (which is optimized for measuring accurate colors between bands), and aperture photometry (flux measured inside circles of varying size).

First, focus on a known part of the simulation to contain a galaxy cluster (also seen in DP0.2 notebook tutorial 03a). Limit the search to contain galaxies using the `i_extendedness` flag (which will exclude point sources), and to those objects that are unique in the objectTable (by setting `detect_isPrimary` = True). Further, identify objects which have been detected at high signal to noise > 20 and whose photometric measurements have not been flagged as having an issue (`i_kronFlux_flag` or `i_cModel_flag` = 0 means the photometry is ok). 

In [ ]:
cluster_RA = 55.7506834813934
cluster_dec = -32.28892993702273


In [ ]:
query = "SELECT obj.objectId, obj.coord_ra, obj.coord_dec, " + \
        "obj.detect_fromBlend, obj.detect_isIsolated, " + \
        "obj.i_blendedness, obj.i_extendedness, " + \
        "obj.i_kronFlux, obj.i_kronFluxErr, obj.i_kronRad, " + \
        "obj.i_cModelFlux, obj.i_cModelFluxErr, obj.i_gaapOptimalFlux, " + \
        "obj.i_kronFlux_flag, obj.i_cModel_flag, " + \
        "obj.shape_xx, obj.shape_xy, obj.shape_yy, " + \
        "obj.i_ap03Flux, obj.i_ap06Flux, obj.i_ap09Flux, obj.i_ap12Flux, " + \
        "obj.i_ap17Flux, obj.i_ap25Flux, obj.i_ap35Flux, obj.i_ap50Flux " + \
        "FROM dp02_dc2_catalogs.Object AS obj " + \
        "WHERE (obj.detect_isPrimary = 1) AND " + \
        "(obj.i_cModelFlux/obj.i_cModelFluxErr > 20) AND " + \
        "(obj.i_extendedness = 1) AND " + \
        "(obj.i_kronFlux_flag = 0) AND (obj.i_cModel_flag = 0) AND " + \
        "CONTAINS(POINT('ICRS', obj.coord_ra, obj.coord_dec), " + \
        "CIRCLE('ICRS',"+str(cluster_RA)+","+str(cluster_dec)+", 0.1)) = 1 "

In [ ]:
job = service.submit_job(query)
job.run()
job.wait(phases=['COMPLETED', 'ERROR'])
print('Job phase is', job.phase)

Print the results of the search query.

In [ ]:
results = job.fetch_result()
print(len(results))
tab = results.to_table()
tab

Below, store the Kron Radius, which is a good proxy for the size of the galaxy light profile. Then convert the fluxes extracted from the objectTable into AB magnitudes.

In [ ]:
i_kronRad = tab['i_kronRad']

cmodel_mag = -2.50 * np.log10(tab['i_cModelFlux']) + 31.4
kron_mag = -2.50 * np.log10(tab['i_kronFlux']) + 31.4
gaap_mag = -2.50 * np.log10(tab['i_gaapOptimalFlux']) + 31.4

ap06_mag = -2.50 * np.log10(tab['i_ap06Flux']) + 31.4
ap09_mag = -2.50 * np.log10(tab['i_ap09Flux']) + 31.4
ap17_mag = -2.50 * np.log10(tab['i_ap17Flux']) + 31.4
ap12_mag = -2.50 * np.log10(tab['i_ap12Flux']) + 31.4
ap35_mag = -2.50 * np.log10(tab['i_ap35Flux']) + 31.4

### 2.3.2 Compare aperture photometry to cModel total fluxes

This section will make several plots that shows how the different photometric measurements compare. Note that cModel is the total flux integrated from a model fit to the galaxy light profile, while Kron, GaaP, and aperture photometry are all measures of light within a fixed aperture.  


#### 2.3.2.1 Aperture photometry

Generally, magnitudes measured using aperture photometry in the LSST pipeline are larger (i.e. fainter) than those measured from cModel, becuase the fixed circular aperture systematically underestimates the flux in the galaxy wings (and the lost flux increases as the intrinsic size of the galaxy increases, e.g. as traced by the Kron radius).

In [ ]:
fig, (ax, ax2) = plt.subplots(ncols=2, nrows=1,
                              width_ratios=[0.8, 0.2], figsize=(10, 6))

ylims = [-1.5, 1.5]

ax.plot(i_kronRad, (cmodel_mag-ap17_mag), '.', alpha=.1,
        label='cModel - 17-pix aperture', color='blue')
ax.plot(i_kronRad, (cmodel_mag-ap12_mag), '.', alpha=.1,
        label='cModel 12-pix aperture', color='green')
ax.plot(i_kronRad, (cmodel_mag-ap09_mag), '.', alpha=.1,
        label='cModel 9-pix aperture', color='orange')
ax.plot(i_kronRad, (cmodel_mag-ap06_mag), '.', alpha=.1,
        label='cModel 6-pix aperture', color='red')

ax2.hist((cmodel_mag-ap17_mag), edgecolor='blue', orientation="horizontal",
         bins=np.linspace(ylims[0], ylims[1], 40), align='mid',
         histtype="step", stacked=True, fill=False)
ax2.hist((cmodel_mag-ap12_mag), edgecolor='green', orientation="horizontal",
         bins=np.linspace(ylims[0], ylims[1], 40), align='mid',
         histtype="step", stacked=True, fill=False)
ax2.hist((cmodel_mag-ap09_mag), edgecolor='orange', orientation="horizontal",
         bins=np.linspace(ylims[0], ylims[1], 40), align='mid',
         histtype="step", stacked=True, fill=False)
ax2.hist((cmodel_mag-ap06_mag), edgecolor='red', orientation="horizontal",
         bins=np.linspace(ylims[0], ylims[1], 40), align='mid',
         histtype="step", stacked=True, fill=False)

ax2.set_ylim(ylims)
ax.axhline(0, linestyle='--')
ax.set_xlabel('Kron Radius [i-band; pixels]')
ax.set_ylabel('cModel mag - Aperture mag')
ax.set_ylim(ylims)
ax.set_xlim([0, 15])
ax.legend()


In [ ]:
fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(7, 6))
ylims = [-1.2, 1.2]
bins = np.arange(16, 27, 1)

ax.plot(cmodel_mag, (cmodel_mag-ap06_mag), '.', alpha=.1,
        label='cModel 6-pix aperture', color='red')
x = cmodel_mag
y = (cmodel_mag-ap06_mag)
bin_mean, bin_edge, binnum = binned_statistic(x, y,
                                              statistic='median', bins=bins)
binctr = bin_edge[:-1] + (bin_edge[1:]-bin_edge[:-1])/2
ax.plot(binctr, bin_mean, color='red', lw=2, label='bin median', zorder=10)

ax.plot(cmodel_mag, (cmodel_mag-ap09_mag), '.', alpha=.1,
        label='cModel 9-pix aperture', color='orange')
x = cmodel_mag
y = (cmodel_mag-ap09_mag)
bin_mean, bin_edge, binnum = binned_statistic(x, y,
                                              statistic='median', bins=bins)
binctr = bin_edge[:-1] + (bin_edge[1:]-bin_edge[:-1])/2
ax.plot(binctr, bin_mean, color='orange', lw=2,
        label='bin median', zorder=10)

ax.plot(cmodel_mag, (cmodel_mag-ap12_mag), '.', alpha=.1,
        label='cModel 12-pix aperture', color='green')
x = cmodel_mag
y = (cmodel_mag-ap12_mag)
bin_mean, bin_edge, binnum = binned_statistic(x, y,
                                              statistic='median', bins=bins)
binctr = bin_edge[:-1] + (bin_edge[1:]-bin_edge[:-1])/2
ax.plot(binctr, bin_mean, color='green', lw=2,
        label='bin median', zorder=10)

ax.plot(cmodel_mag, (cmodel_mag-ap17_mag), '.', alpha=.1,
        label='cModel 17-pix aperture', color='blue')
x = cmodel_mag
y = (cmodel_mag-ap17_mag)
bin_mean, bin_edge, binnum = binned_statistic(x, y,
                                              statistic='median', bins=bins)
binctr = bin_edge[:-1] + (bin_edge[1:]-bin_edge[:-1])/2
ax.plot(binctr, bin_mean, color='blue', lw=2,
        label='bin median', zorder=10)

ax.axhline(0, linestyle='--')
ax.set_xlabel('cModel Magnitude')
ax.set_ylabel('cModel mag - Aperture mag')
ax.set_ylim([-1, 1])
ax.legend()


These two figures show that the aperture photometry typically under-estimates the flux relative to the total flux estimated using cModel. As expected, there is a general trend for larger apertures to get closer to the total flux from cModel for large galaxies (i.e. whose Kron Radius is larger). There is also a general trend for the aperture photometry to be less discrepant at fainter magnitudes, since faint galaxies tend to be small.


### 2.3.2 Compare Kron and GaaP photometry to cModel

In the next cell, compare the cModel instead to the Kron and GaaP measures.

In [ ]:
fig, (ax, ax2) = plt.subplots(ncols=2, nrows=1,
                              width_ratios=[0.8, 0.2], figsize=(10, 6))
ylims = [-2, 2]

ax.plot(i_kronRad, (cmodel_mag-kron_mag), '.', alpha=.1,
        label='cModel-Kron')
ax.plot(i_kronRad, (cmodel_mag-gaap_mag), '.', alpha=.1,
        label='cModel - gaapOptimal')

ax2.hist((cmodel_mag-kron_mag), orientation="horizontal",
         bins=np.linspace(ylims[0], ylims[1], 40), align='mid',
         histtype="step", stacked=True, fill=False)
ax2.hist((cmodel_mag-gaap_mag), orientation="horizontal",
         bins=np.linspace(ylims[0], ylims[1], 40), align='mid',
         histtype="step", stacked=True, fill=False)

ax2.set_ylim(ylims)
ax.axhline(0, linestyle='--')
ax.set_xlabel('Kron Radius [i-band; pixels]')
ax.set_ylabel('cModel mag - Other mag')
ax.set_ylim(ylims)
ax.set_xlim([0, 15])
ax.legend()


In [ ]:
fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(7, 6))
ylims = [-1.2, 1.2]

ax.plot(cmodel_mag, (cmodel_mag-kron_mag), '.', alpha=.1,
        label='cModel-Kron', color='blue')
x = cmodel_mag
y = (cmodel_mag-kron_mag)
bin_mean, bin_edge, binnum = binned_statistic(x, y,
                                              statistic='median', bins=bins)
binctr = bin_edge[:-1] + (bin_edge[1:]-bin_edge[:-1])/2
ax.plot(binctr, bin_mean, color='blue', lw=2, label='bin median', zorder=10)

ax.plot(cmodel_mag, (cmodel_mag-gaap_mag), '.', alpha=.1,
        label='cModel - gaapOptimal', color='orange')

x = cmodel_mag
y = (cmodel_mag-gaap_mag)
bin_mean, bin_edge, binnum = binned_statistic(x, y,
                                              statistic='median', bins=bins)
binctr = bin_edge[:-1] + (bin_edge[1:]-bin_edge[:-1])/2
ax.plot(binctr, bin_mean, color='orange', lw=2, label='bin median', zorder=10)

ax.axhline(0, linestyle='--')
ax.set_xlabel('cModel Magnitude')
ax.set_ylabel('cModel mag - Other mag')
ax.set_ylim([-1, 1])
ax.legend()

The above plots show the parameter space where Kron fluxes do not correspond well with total fluxes (as measured by cModel). This indcates that Kron fluxes will underestimate the total fluxes in the regime of bright magnitudes and large Kron radius. Generally, cModel is recommended over Kron.

As explained in Section 1, GaaP fluxes are not meant to measure total fluxes (from these plots it is clear there is a strong discrepancy with cModel) but GaaP measurements are the best and most robust for measuring robust integrated colors between different filters. Thus GaaP measurements should be used for Spectral Energy Distribution (SED) or photometric redshift modeling, or for color selections.

### 3. Visualize photometric measurements

Here make a cutout of a large galaxy in the cluster and compare the photometric aperture sizes of the different photometric measurements. The cell below also demonstrates how to reconstruct the Kron aperture using the corresponding shape parameters in the objectTable. The position angle `theta` is defined in radians counterclockwise from the x-axis. This cell will plot for an elongated edge-on galaxy (`objectId` of 1909948454470183768) to ensure that the visualization emphasizes the difference between the elliptical Kron aperture to the circular aperture. 

In [ ]:
wh = np.where((tab['objectId'] == 1909948454470183768))[0]
indx = 0

#  Alternatively, pick a different bright object that has large size
#  wh = np.where((tab['i_kronRad'] > 8) & (cmodel_mag < 20))[0]

arcsec_per_pix = 0.2  # this is the pixel scale

mem = make_image_cutout_new(service, tab['coord_ra'][wh][indx],
                            tab['coord_dec'][wh][indx], cutout_size=0.005)

cutout = ExposureF(mem)

axes = ellipses.Axes(ellipses.Quadrupole(tab['shape_xx'][wh][indx],
                                         tab['shape_yy'][wh][indx],
                                         tab['shape_xy'][wh][indx]))
Rmaj = axes.getA()
Rmin = axes.getB()
theta = axes.getTheta()

plt.subplot(projection=WCS(cutout.getWcs().getFitsMetadata()))
extent = (cutout.getBBox().beginX, cutout.getBBox().endX,
          cutout.getBBox().beginY, cutout.getBBox().endY)

plt.imshow(cutout.image.array, vmin=.1, vmax=0.8,
           extent=extent, origin='lower', cmap='gray')

coord = SkyCoord(ra=tab['coord_ra'][wh][indx]*u.degree,
                 dec=tab['coord_dec'][wh][indx]*u.degree, frame='icrs')

aperture = SkyCircularAperture(coord, r=17 * arcsec_per_pix * u.arcsec)
pix_aperture = aperture.to_pixel(WCS(cutout.getWcs().getFitsMetadata()))
pix_aperture.plot(color='r', lw=3, label='17 pix aperture')

aperture = SkyCircularAperture(coord, r=35 * arcsec_per_pix * u.arcsec)
pix_aperture = aperture.to_pixel(WCS(cutout.getWcs().getFitsMetadata()))
pix_aperture.plot(color='g', lw=3, label='35 pix aperture')

aperture = SkyEllipticalAperture(coord,
                                 2.5 * Rmaj * arcsec_per_pix * u.arcsec,
                                 2.5 * Rmin * arcsec_per_pix * u.arcsec,
                                 theta=(np.pi/2 + theta) * u.rad)

pix_aperture = aperture.to_pixel(WCS(cutout.getWcs().getFitsMetadata()))
pix_aperture.plot(color='b', lw=3, label='Kron')
plt.legend()

print("Kron magnitude = ", kron_mag[wh][indx])
print("17-pix Aperture mag = ", ap17_mag[wh][indx])
print("35-pix Aperture mag = ", ap35_mag[wh][indx])
print("cModel mag = ", cmodel_mag[wh][indx])


The above figure shows more clearly that there is some fraction of the galaxy's light excluded by each of these photometric apertures. Further, printing the different photometry measurements illustrates how these assumed apertures will change the measured flux (compared to the total flux measured by cModel, which approximates the total magnitude using sersic models for the light profile).  

## 4. Application of aperture photometry: radial profile

A science application for the aperture photometry is easy visualization of the radial profile of galaxies. In the cell below, make this plot for both a large galaxy and a small galaxy (of similar brightness, cModel magnitude ~ 20). Dividing the aperture flux by the surface area of the aperture yields the surface brightness, which can be plotted as a function of radius from the center of the galaxy to compare radial light profiles.

In [ ]:
#  pick a bright object with large size
wh = np.where((tab['i_kronRad'] > 8) & (cmodel_mag > 19.9)
              & (cmodel_mag < 20.1))[0]
indx = 2

rad = np.array([3, 6, 9, 12, 17, 25, 35, 50]) * arcsec_per_pix
area = np.pi * rad**2
profile = np.array([tab['i_ap03Flux'][wh][indx], tab['i_ap06Flux'][wh][indx],
                    tab['i_ap09Flux'][wh][indx], tab['i_ap12Flux'][wh][indx],
                    tab['i_ap17Flux'][wh][indx], tab['i_ap25Flux'][wh][indx],
                    tab['i_ap35Flux'][wh][indx],
                    tab['i_ap50Flux'][wh][indx]]) / area

plt.plot(rad, profile,
         label='Large Kron Radius R='+str(i_kronRad[wh][indx]*arcsec_per_pix))
plt.xlabel('Aperture Radius [arcsec]')
plt.ylabel(r'Surface Brightness [nJy arcsec$^{-2}$]')

# pick an object of similar magnitude that has a smaller size
wh2 = np.where((tab['i_kronRad'] < 8) & (tab['i_kronRad'] > 5)
               & (cmodel_mag > 19.9) & (cmodel_mag < 20.1))[0]

indx = 1

profile = np.array([tab['i_ap03Flux'][wh2][indx], tab['i_ap06Flux'][wh2][indx],
                    tab['i_ap09Flux'][wh2][indx], tab['i_ap12Flux'][wh2][indx],
                    tab['i_ap17Flux'][wh2][indx], tab['i_ap25Flux'][wh2][indx],
                    tab['i_ap35Flux'][wh2][indx],
                    tab['i_ap50Flux'][wh2][indx]])/area

plt.plot(rad, profile,
         label='Small Kron Radius R='+str(i_kronRad[wh2][indx]*arcsec_per_pix))
plt.legend()
plt.yscale('log')

## 5. Exercise for the learner

Compare the `<f>_free_cModelFlux` measurements to `<f>_cModelFlux` in the filters that are not the reference band where the `<f>_cModelFlux` was measured (i.e. `refBand`). Investigate how leaving the cModel measurements free differs from the one measured with parameters fixed to the `refBand`, as a function of decreasing signal to noise. 